# 0. Importing Packages

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import timedelta
import isodate

In [ ]:
#data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

In [ ]:
#summary tools for enhanced EDA
!pip install summarytools
from summarytools import dfSummary

#ignore warnings
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)

# 1. Load Dataset

In [ ]:
df = pd.read_csv('E:/OneDrive - UTS/Sem 4 (Spring 2024)/Advanced ML Applications/Assignment 3/adv_mla_at3/data/raw/df_airport.csv', low_memory=False)

# 2. Data Exploration and Transformation

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
dfSummary(df)

In [ ]:
# unique values for desitination airport
unique_airports = df['destinationAirport'].unique()
print(unique_airports)

**At a quick glance here are the observations made:**

dimenstion:  9,467,508 x 23
1. There are 4 starting airport (SFO, ORD, PHL, OAK) since I am working on this subset 
2. There are 16 destination airports. 
3. Most of the tickets are not Basic Economy (96.6%)
4. Almost all the tickets are non refundable.
5. The median total fare is about 408.6 dollars with the highest being at 8260 dollars. 
6. Median distance travelled is about 1784.0 miles with about 7% missing data.
7. If there are transits, the segments are broken down with || for some of the features stating Departure and Arrival Airport, Airline name, airline model, duration, distance and cabin code.
8. Data quality: there are 6,311,672 duplicate values


In [ ]:
print(df.dtypes)

In [ ]:
#convert it to datetime 
df['searchDate'] = pd.to_datetime(df['searchDate'], errors='coerce')

# Find the minimum and maximum dates
min_date = df['searchDate'].min()
max_date = df['searchDate'].max()

# Display the results
print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

After converting the searchDate from object to Datetime, it is seen that the minimum date for searching a flight is on 16/4/2022 and the maximum date is 19/5/2022

In [ ]:
#convert it to datetime 
df['flightDate'] = pd.to_datetime(df['flightDate'], errors='coerce')

# Find the minimum and maximum dates
min_date = df['flightDate'].min()
max_date = df['flightDate'].max()

# Display the results
print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")


After converting the flightDate from object to Datetime, it is seen that the minimum date for searching a flight is on 17/4/2022 and the maximum date is 17/7/2022. It means that some customers looked for flight on the next day from 16/4/2022.

In [ ]:
df_prep = df.copy()

The total duration is in ISO 8601 format and it will be converted to hours

In [ ]:
df_prep['travelDurationHours'] = df_prep['travelDuration'].apply(
    lambda x: isodate.parse_duration(x).total_seconds() / 3600
)

In [ ]:
df_prep.head()

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
df_prep['travelDurationHours'].describe()

Based on this new feature, we can see that the median value for the flight duration is about 7.5 hours, minimum value is less than an hour and max is about 39.5 hours

### duplicate handling

In [ ]:
# Count the total number of duplicate rows
total_duplicates = df_prep.duplicated().sum()

# Display the total count
print(f"Total number of duplicate rows: {total_duplicates}")

In [ ]:
#let's verify if all the columns match in some of the duplicates

# Find all duplicate rows (including the first occurrence)
duplicates = df_prep[df_prep.duplicated(keep=False)]

# Sort the duplicates to group identical rows together
grouped_duplicates = duplicates.sort_values(list(df_prep.columns))

# Display the grouped duplicate observations
grouped_duplicates.head()


In [ ]:
grouped_duplicates.tail()

Therefore we can conclude that there are actually duplicate observations and it's a significant number. 6,311,672/9,467,508 are duplicates which is 2/3 of the dataset.

In [ ]:
#dropping duplicates
df_prep = df_prep.drop_duplicates()

In [ ]:
# Count the total number of duplicate rows
total_duplicates = df_prep.duplicated().sum()

# Display the total count
print(f"Total number of duplicate rows: {total_duplicates}")

In [ ]:
df_prep.shape

It can be seen that the dataframe is now 1/3 of the initial size.

### Data Visualisation

##### Let's look at the distribution of the target variable

In [ ]:
fig = px.histogram(
    df_prep, 
    x='totalFare', 
    nbins=100,  
    title='Distribution of Total Fare',
    labels={'totalFare': 'Total Fare'},
    width=800, 
    height=500
)

# Limit the x-axis range to focus on the bulk of the data
fig.update_layout(xaxis=dict(range=[0, 1000]))  

fig.show()


From the range and spread above, we can see that the totalFare values mainly fall bewteen 0 and 1000 dollars. The histogram is slighly right skewed and indicating that most fares are concentrated between 200 and 600. As fare amount increases, the frequency decreases. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

# Create the box plot 
sns.boxplot(y=df_prep['totalFare'])
plt.title('Box Plot of Total Fare')
plt.ylabel('Total Fare')

# Show the plot
plt.show()


From the box plot, we can see that the median fare is at the lower end (408). There are several datapoints which are beyond the upper whisker that go beyond 2000 dollars with some reaching as high as 8000 dollars.

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df_prep['totalFare'].quantile(0.25)
Q3 = df_prep['totalFare'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the upper and lower bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df_prep[(df_prep['totalFare'] < lower_bound) | (df_prep['totalFare'] > upper_bound)]

# Count the number of outliers
outlier_count = outliers.shape[0]

# Display the result
print(f"Number of outliers in totalFare: {outlier_count}")


We can see that there are about 42,850 rows which are outliers (1.35% of data). Since my objective is to predict typical fares, I will remove the outliers.

In [ ]:
#REMOVE OUTLIERS
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df_prep['totalFare'].quantile(0.25)
Q3 = df_prep['totalFare'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the upper and lower bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers from the DataFrame
df_prep = df_prep[(df_prep['totalFare'] >= lower_bound) & (df_prep['totalFare'] <= upper_bound)]

# Verify the new size of the DataFrame
print(f"Number of rows after removing outliers: {df_prep.shape[0]}")


##### Let's look at the frequency of starting and destination airports

In [ ]:
#  Count occurrences of each starting airport
airport_counts = df_prep['startingAirport'].value_counts().reset_index()
airport_counts.columns = ['startingAirport', 'Count']

# Create the bar chart 
fig = px.bar(
    airport_counts,
    x='startingAirport',
    y='Count',
    title='Count of Flights by Starting Airport',
    labels={'startingAirport': 'Starting Airport', 'Count': 'Number of Flights'},
    width=600,  
    height=500  
)

fig.update_traces(width=0.5)  

# Plot
fig.show()

In [ ]:
# Count occurrences of each destination airport
airport_counts = df_prep['destinationAirport'].value_counts().reset_index()
airport_counts.columns = ['destinationAirport', 'Count']

# Create the bar chart 
fig = px.bar(
    airport_counts,
    x='destinationAirport',
    y='Count',
    title='Count of Flights by Destination Airport',
    labels={'destinationAirport': 'Destination Airport', 'Count': 'Number of Flights'},
    width=600,  
    height=500  
)


fig.update_traces(width=0.5)  

# Plot
fig.show()

We can see that the highest frequency of flights took off from SFO and the least from OAK. The highest frequency destination airport was LGA and the least was OAK.

##### Distribution of total travel distance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Set plot size
plt.figure(figsize=(10, 6))

# Step 2: Create a histogram using Seaborn
sns.histplot(df_prep['totalTravelDistance'], bins=50, kde=True)

# Step 3: Add title and labels
plt.title('Distribution of Total Travel Distance')
plt.xlabel('Total Travel Distance')
plt.ylabel('Frequency')

# Step 4: Show the plot
plt.show()


The distribution of totalTravelDistance shows a multi-modal pattern with several peaks at different intervals (e.g., around 500, 1000, 2500). Based on mulitple peaks, we can deduce that there are several distinct groups of travel distances, suggesting flights of varying lengths.

##### Look at the first segment distribution of flights with more than one layovers

In [ ]:
# Extract the first segment (before the first '|') from 'segmentsCabinCode'
df_prep['firstSegmentCabinCode'] = df_prep['segmentsCabinCode'].str.split('|').str[0]

# Count the frequency of the first segments
first_segment_counts = df_prep['firstSegmentCabinCode'].value_counts()

# Display the frequency
print(first_segment_counts)


We can see that majority of the observations have coach as the cabin code for the first segment of the flight.

##### Look at the first segment distribution of airlines with more than one layovers

In [ ]:
#Extract the first airline name (before the first '|') from 'segmentsAirlineName'
df_prep['firstSegmentAirline'] = df_prep['segmentsAirlineName'].str.split('|').str[0]

# Count the frequency of the first airline names
first_airline_counts = df_prep['firstSegmentAirline'].value_counts()

# Display the frequency
print(first_airline_counts)


In [ ]:
# Set plot size
plt.figure(figsize=(10, 6))

# Create a histogram with KDE overlay
sns.histplot(df_prep['travelDurationHours'], bins=30, kde=True)

# Add title and labels
plt.title('Distribution of Travel Duration in Hours')
plt.xlabel('Travel Duration (Hours)')
plt.ylabel('Frequency')

# Show the plot
plt.show()


From the chart above, we can see that majority of the flights have a shorter travel duration (2-10 hours) with  a decreasing number of longer flights. There are some flights that extend up to 40 hours but very rate. 

In [ ]:
#save to interim folder
file_path = 'E:/OneDrive - UTS/Sem 4 (Spring 2024)/Advanced ML Applications/Assignment 3/adv_mla_at3/data/interim/df_airport_farhan_faiyaz.csv'
df_prep.to_csv(file_path, index=False)